# Who is the Most Clutch Player from the 2025 MLB Season?
## sxm4bz
## Ryan Colavita
## Notebook 1 - Establishing and Reading the Data

In baseball, a player steps up to the plate and for the entire at-bat, is almost completely on their own. There is nobody else helping that player hit the ball. The player is isolated at the plate which makes big plays late in the game hard to do. Players that can make big plays late in the game have a "clutch" talent and don't break under pressure. This makes them very valuable to the team. Which poses the question, who is the most clutch player in the MLB? 

In this notebook, I read through all of my data and save it as one file. Since there is so much data that is tracked in MLB games, it is not available to download as one file. However, it is able to be downloaded by month since the file size isn't as big that way. Below, there is a loop that runs through the files of monthly MLB game data and combines it into one file called "savant_data." While combining the files, I have also isolated the necessary columns for calculating the most clutch player in order to save memory. 

The Data Source: I got my initial data from "Baseball Savant" which is ran by the MLB. The MLB has a public website made for anyone looking to explore MLB data. I got the data by downloading the 2025 MLB season by month from the website. The features of the data are produced in a table below. 

In [3]:
#Combining the monthly files into one file

import pandas as pd
import glob
import os

FILE_PATTERN = 'savant_*.csv'
OUTPUT_FILE_NAME = 'savant_data.csv'

all_data = []

print(f"Searching for files matching pattern: {FILE_PATTERN} in the current directory...")

csv_files = glob.glob(FILE_PATTERN)

if not csv_files:
    print(f"ERROR: No files found matching '{FILE_PATTERN}'. Please check your file names.")
else:
    print(f"Found {len(csv_files)} files. Starting concatenation...")

    for filename in csv_files:
        try:
            COLUMNS_TO_LOAD = [
                'player_name',
                'events',
                'delta_home_win_exp',
                'inning_topbot'
            ]
            
            df = pd.read_csv(filename, usecols=COLUMNS_TO_LOAD, low_memory=False)
            all_data.append(df)
            print(f"  - Successfully loaded {filename} ({len(df):,} rows)")
            
        except Exception as e:
            print(f"  - WARNING: Could not read {filename}. Skipping. Error: {e}")

    if all_data:
        data_combined = pd.concat(all_data, ignore_index=True)
        
        data_combined.to_csv(OUTPUT_FILE_NAME, index=False)
        
        print(f"SUCCESS! All files combined.")
        print(f"Total Rows in Final Dataset: {len(data_combined):,}")
        print(f"Saved to: {OUTPUT_FILE_NAME}")
    else:
        print("\nFailed to load any files. Cannot create combined dataset.")

Searching for files matching pattern: savant_*.csv in the current directory...
Found 7 files. Starting concatenation...
  - Successfully loaded savant_september.csv (25,000 rows)
  - Successfully loaded savant_april.csv (25,000 rows)
  - Successfully loaded savant_may.csv (25,000 rows)
  - Successfully loaded savant_august.csv (25,000 rows)
  - Successfully loaded savant_june.csv (25,000 rows)
  - Successfully loaded savant_july.csv (25,000 rows)
  - Successfully loaded savant_data.csv (300,000 rows)
SUCCESS! All files combined.
Total Rows in Final Dataset: 450,000
Saved to: savant_data.csv


In [4]:
#Testing to make sure the required columns can be used from the combined file. 

FILE_PATH = 'savant_data.csv' 

COLUMNS_TO_LOAD = [
    'player_name',
    'events',
    'delta_home_win_exp', #New WPA column
    'inning_topbot'       
]

print(f"Reading data from: {FILE_PATH}...")

try:
    data_raw = pd.read_csv(FILE_PATH, usecols=COLUMNS_TO_LOAD, low_memory=False)
except FileNotFoundError:
    print(f"ERROR: File not found at {FILE_PATH}. Please check the path and file name.")
    data_raw = None 
except ValueError as e:
    print(f"ERROR: Required columns are missing. Please ensure the file is the pitch-by-pitch Statcast data.")
    print(f"Original Pandas Error: {e}")
    data_raw = None

if data_raw is not None:
    print(f"Initial Data Description")
    print(f"Total Rows Loaded: {len(data_raw):,}")

    print("\nFirst 5 Rows:")
    print(data_raw.head().to_markdown())
    print("\nDataFrame Structure (Features):")
    data_raw.info()
else:
    print("\nData loading failed. Cannot proceed with analysis.")

Reading data from: savant_data.csv...
Initial Data Description
Total Rows Loaded: 450,000

First 5 Rows:
|    | player_name    | events    | inning_topbot   |   delta_home_win_exp |
|---:|:---------------|:----------|:----------------|---------------------:|
|  0 | McCarthy, Jake | field_out | Top             |                0.008 |
|  1 | McCarthy, Jake | nan       | Top             |                0.003 |
|  2 | McCarthy, Jake | nan       | Top             |                0.003 |
|  3 | McCarthy, Jake | nan       | Top             |                0.003 |
|  4 | McCarthy, Jake | nan       | Top             |                0.003 |

DataFrame Structure (Features):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   player_name         450000 non-null  object 
 1   events              116013 non-null  object 
 2   inning_t

In [19]:
#COLS Table
import pandas as pd
import numpy as np

FILE_PATH = 'savant_data.csv'

try:
    data_full = pd.read_csv(FILE_PATH, low_memory=False)
    
except FileNotFoundError:
    print(f"ERROR: File not found at {FILE_PATH}. Please ensure it is in the correct directory.")
    exit() 
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    exit()

cols_table = pd.DataFrame({
    'Feature Name': data_full.columns,
    'Data Type': data_full.dtypes.values
})

cols_table['Non-Null Count'] = data_full.count().values
total_rows = len(data_full)
cols_table['Non-Null %'] = (cols_table['Non-Null Count'] / total_rows * 100).round(2)

feature_descriptions = {
    'player_name': 'Name of the batter (hitter).',
    'events': 'The resulting event of the plate appearance (e.g., single, strikeout, walk). Used to identify PA-ending pitches.',
    'delta_home_win_exp': 'The change in Home Team Win Expectancy. This column is used as the **Win Probability Added (WPA)**.',
    'inning_topbot': 'Indicates if it is the Top (away team batting) or Bot (home team batting) of the inning, crucial for assigning WPA to the batter.',
    'game_date': 'The date the game was played.',
    'release_speed': 'The velocity of the pitch in mph.',
    'release_pos_x': 'Horizontal distance from the catcher’s perspective in feet at the time of pitch release.',
    'pitch_type': 'Designation for the type of pitch thrown (e.g., FF, SL, CH).',
    'at_bat_number': 'The unique number of the at-bat within the game, used to count plate appearances (PA).'
}

cols_table['Description'] = cols_table['Feature Name'].apply(
    lambda x: feature_descriptions.get(x, 'Statcast variable describing pitch, trajectory, or field geometry.')
)

cols_table = cols_table.sort_values(by='Feature Name').reset_index(drop=True)

def generate_example(dtype):
    if 'int' in str(dtype):
        return '641598 (ID)'
    elif 'float' in str(dtype):
        return '0.007'
    elif 'object' in str(dtype):
        return 'single'
    else:
        return 'N/A'

cols_table['Example Value'] = cols_table['Data Type'].apply(generate_example)

cols_table.loc[cols_table['Feature Name'] == 'delta_home_win_exp', 'Example Value'] = '0.045 (WPA)'
cols_table.loc[cols_table['Feature Name'] == 'events', 'Example Value'] = 'home_run'
cols_table.loc[cols_table['Feature Name'] == 'inning_topbot', 'Example Value'] = 'Bot'

final_cols_df = cols_table[['Feature Name', 'Data Type', 'Description', 'Example Value', 'Non-Null Count', 'Non-Null %']]
final_cols_df

,Feature Name,Data Type,Description,Example Value,Non-Null Count,Non-Null %
0,delta_home_win_exp,float64,The change in Home Team Win Expectancy. This c...,0.045 (WPA),450000,100.00
1,events,object,The resulting event of the plate appearance (e...,home_run,116013,25.78
2,inning_topbot,object,Indicates if it is the Top (away team batting)...,Bot,450000,100.00
3,player_name,object,Name of the batter (hitter).,single,450000,100.00
